In [63]:
import numpy as np

In [120]:
# initialize NN and set parameters
input_size = 9 
output_size = 9
epsilon = 0.3 # Probabilty to take random actions
gamma = 0.7 # Discount factor
learning_rate = 1e-3

W1 = np.random.randn(input_size, output_size) / np.sqrt(output_size)
b1 = np.random.randn(output_size) / np.sqrt(output_size)

def forward_pass(x):
    return np.dot(x,W1) + b1

def backward_pass(x, y_diff):
    

    pass

def get_propabilities(x):
    return np.exp(x)/np.sum(np.exp(x))


print(f'{W1=}\n {b1=}') 

W1=array([[-0.15893564,  0.13024658, -0.11753192, -0.14273552, -0.08926084,
        -0.38934029,  0.04215685,  0.33183259, -0.01957468],
       [ 0.01818512, -0.17570159,  0.71097321, -0.05210045, -0.03237064,
        -0.46694079,  0.03334698, -0.26382745,  0.30948366],
       [ 0.46908631,  0.73161488, -0.07080723, -0.58263723, -0.21744845,
        -0.02169667, -0.13139818, -0.0646271 ,  0.34550623],
       [-0.13388643, -0.0874081 , -0.29522438,  0.61991822, -0.25045293,
        -0.29769936, -0.21374882, -0.1361205 , -0.2994686 ],
       [ 0.65545645,  0.16517154,  0.25986713, -0.04452332,  0.19452173,
        -0.30758339, -0.46939769,  0.26410541, -0.34668873],
       [-0.16176667, -0.11509238,  0.16555023,  0.12605055,  0.2009971 ,
         0.07635315,  0.0564335 , -0.67768758, -0.14939983],
       [ 0.57444312,  0.12010707, -0.07575461,  0.59967204,  0.19016486,
        -0.0134451 ,  0.10441614, -0.49036446, -0.26923222],
       [ 0.16593601, -0.18524726, -0.04499084, -0.52225069,

In [65]:
dummy_x = np.random.randn(9)

logits = forward_pass(dummy_x)
probabilities = get_propabilities(logits)
print(f'{logits=}\n {probabilities=}')

logits=array([-0.86497345, -0.17375046,  1.28968619, -0.81365114,  0.08738364,
        0.96772606, -0.4105061 ,  0.460592  , -0.62226655])
 probabilities=array([0.0355484 , 0.07096012, 0.3066033 , 0.03742045, 0.09213475,
       0.22220371, 0.0560006 , 0.13381531, 0.04531336])


In [66]:
def print_board(s):
    x = ['-', 'x', 'o']
    print(f' [{x[s[0]]} {x[s[1]]} {x[s[2]]}] \n [{x[s[3]]} {x[s[4]]} {x[s[5]]}] \n [{x[s[6]]} {x[s[7]]} {x[s[8]]}] \n')

def get_discounted_rewards(actions, reward):
    rewards = [reward * gamma**i for i in range(len(actions))]
    return rewards[::-1]    

In [67]:
# define game loop. Start with the Ai always starting and the opponent going second
ai_token = 1
opponent_token = 2

def take_random_action(board, token):
    free_indices = [i for i, val in enumerate(board) if val == 0]
    action = np.random.choice(free_indices)
    board[action] = token
    return action, board

def take_action(board, logits, token):
    if epsilon > np.random.random_sample():
        action, board = take_random_action(board, token)
    else:
        free_indices = [i for i, val in enumerate(board) if val == 0]
        l_free = logits[free_indices] # get logits of possible moves
        p = get_propabilities(l_free)
        action = np.random.choice(free_indices, p=p)
        board[action] = token
    return action, board

def check_winner(board):

    winning_combinations = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),
        (0, 3, 6), (1, 4, 7), (2, 5, 8),
        (0, 4, 8), (2, 4, 6)
    ]
    for combo in winning_combinations:
        if board[combo[0]] == board[combo[1]] == board[combo[2]] != 0:
            return board[combo[0]] 
    return 0

def get_reward(winner):
    if winner == ai_token:
        return 1
    elif winner == opponent_token:
        return -1
    else:
        return 0

def play_game():
    board = [0] * 9
    boards_arr = []
    actions_arr = []
    logits_arr = []
    for t in range(9):
        if t%2 == 0:
            boards_arr.append(board.copy())
            logits = forward_pass(board)
            action, board = take_action(board, logits, ai_token)
            actions_arr.append(action) 
            logits_arr.append(logits)
        else:
            action, board = take_random_action(board, opponent_token)   
        winner = check_winner(board)
        reward = get_reward(winner)
        if reward:
            break    
    return boards_arr, actions_arr, logits_arr, reward

boards, actions, logits, reward = play_game()

In [68]:
rewards = get_discounted_rewards(actions, reward)
print(rewards)

[0.48999999999999994, 0.7, 1.0]


In [69]:
print(f'{actions=} \n {logits=}')

actions=[2, 6, 4] 
 logits=[array([ 0.06571969, -0.08514417,  0.81530028, -0.37766099, -0.16058463,
       -0.34139338,  0.39587798,  0.51943669, -0.60444246]), array([ 1.52537055, -0.03882505,  0.69816536, -0.36037973,  0.24198805,
       -1.57083642,  1.1833322 ,  0.40794881, -0.15951308]), array([ 1.88353529, -1.69761012,  0.39188013, -0.5382177 ,  1.54938948,
       -1.37590337,  2.78663691,  0.36361585, -1.01436872])]


In [104]:
episodes = 2 # Do 10 training episodes
n = 10 # play 1000 games

def get_ydiff(logits, actions):
    y_diff = []
    for lg, a in zip(logits, actions):
        act_arr = [0] * 9
        act_arr[a] = 1
        y_diff.append(act_arr - get_propabilities(lg))
    return y_diff

for e in range(episodes):
    X, y_diff, R = [], [], [] # X : Training Data of board states, y_diff Difference between action taken and output probabilities
    # R: Rewards for the (X,y) samples
    for i in range(n):
        boards, actions, logits, reward = play_game()
        rewards = get_discounted_rewards(actions, reward)
        diff = get_ydiff(logits, actions)

        X.append(boards)
        y_diff.append(diff)
        R.append(rewards)

    # reduce dimensionality to avoid problems with different game length
    Xeps = np.array([b for g in X for b in g]) 
    yeps = np.array([y for g in y_diff for y in g])
    Reps = np.array([r for g in R for r in g])

    # normalize the rewards
    Reps -= np.mean(Reps)
    Reps /= np.std(Reps)

    yeps *= Reps[:, np.newaxis] # mutiply gradient and reward

    dW, db = backward_pass(yeps)
    W1 += learning_rate*dW
    b1 += learning_rate*db

print(Xeps.shape)
print(yeps.shape)
print(Reps.shape)


(41, 9)
(41, 9)
(41,)


In [117]:
print(Reps)
R_n = Reps - np.mean(Reps)
R_n /= np.std(R_n)
print(R_n)

[ 0.343   0.49    0.7     1.     -0.343  -0.49   -0.7    -1.      0.2401
  0.343   0.49    0.7     1.      0.49    0.7     1.      0.      0.
  0.      0.      0.      0.343   0.49    0.7     1.     -0.343  -0.49
 -0.7    -1.     -0.343  -0.49   -0.7    -1.      0.343   0.49    0.7
  1.      0.343   0.49    0.7     1.    ]
[ 0.26306186  0.504496    0.84940192  1.34212465 -0.86363079 -1.10506493
 -1.44997084 -1.94269357  0.09405797  0.26306186  0.504496    0.84940192
  1.34212465  0.504496    0.84940192  1.34212465 -0.30028446 -0.30028446
 -0.30028446 -0.30028446 -0.30028446  0.26306186  0.504496    0.84940192
  1.34212465 -0.86363079 -1.10506493 -1.44997084 -1.94269357 -0.86363079
 -1.10506493 -1.44997084 -1.94269357  0.26306186  0.504496    0.84940192
  1.34212465  0.26306186  0.504496    0.84940192  1.34212465]
